project 1 - discovery

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from scipy.stats import linregress
from pprint import pprint
from sodapy import Socrata


In [16]:
# Authenticated client request
request_url = Socrata("data.sonomacounty.ca.gov", "svXzQDyVcEYVmqw6LDWsmj7VP",
                 username="rondajoy@gmail.com", password="Flwrchild75", timeout=500)

# return as JSON from API then converted to Python list of dictionaries by sodapy
results = request_url.get("924a-vesw", limit=26000)

# Convert to pandas DataFrame
results_df = pd.DataFrame(results)
results_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25285 entries, 0 to 25284
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   name                         18658 non-null  object
 1   type                         25285 non-null  object
 2   breed                        25285 non-null  object
 3   color                        25285 non-null  object
 4   sex                          25285 non-null  object
 5   size                         25250 non-null  object
 6   date_of_birth                18971 non-null  object
 7   impound_number               25285 non-null  object
 8   kennel_number                25261 non-null  object
 9   id                           25285 non-null  object
 10  intake_date                  25285 non-null  object
 11  outcome_date                 25113 non-null  object
 12  days_in_shelter              25285 non-null  object
 13  intake_type                  25

---

In [17]:
# delete unnecessary column
del results_df[':@computed_region_dig5_f3vy']
del results_df['intake_total']

# transform data type for all date columns
results_df['date_of_birth'] = pd.to_datetime(results_df.date_of_birth)
results_df['intake_date'] = pd.to_datetime(results_df.intake_date)
results_df['outcome_date'] = pd.to_datetime(results_df.outcome_date)

# Create date features for grouping
results_df['yr_intake']=results_df.intake_date.dt.year
results_df['mon_intake']=results_df.intake_date.dt.month

results_df['yr_outcome']=results_df.outcome_date.dt.year
results_df['mon_outcome']=results_df.outcome_date.dt.month

# Check for outliers via groupby year
results_df['yr_intake'].unique()
results_df.groupby('yr_intake').count()


,name,type,breed,color,sex,size,date_of_birth,impound_number,kennel_number,id,...,outcome_subtype,intake_condition,outcome_condition,intake_jurisdiction,outcome_jurisdiction,zip_code,location,mon_intake,yr_outcome,mon_outcome
yr_intake,,,,,,,,,,,,,,,,,,,,,
2013,108,120,120,120,120,120,107,120,120,120,...,120,120,120,120,101,101,101,120,120,120
2014,2500,3216,3216,3216,3216,3208,2705,3216,3216,3216,...,3213,3216,3205,3216,2774,2758,2758,3216,3216,3216
2015,2430,3199,3199,3199,3199,3195,2510,3199,3197,3199,...,3195,3199,3194,3199,2740,2718,2718,3199,3197,3197
2016,2458,3235,3235,3235,3235,3231,2494,3235,3233,3235,...,3225,3235,3228,3235,2748,2742,2742,3235,3233,3233
2017,2323,3154,3154,3154,3154,3150,2337,3154,3154,3154,...,3089,3154,3150,3154,2738,2689,2689,3154,3154,3154
2018,2168,3081,3081,3081,3081,3077,2195,3081,3081,3081,...,3058,3081,3023,3081,2669,2673,2673,3081,3081,3081
2019,2038,2787,2787,2787,2787,2781,1976,2787,2787,2787,...,2746,2787,2714,2787,2457,2482,2482,2787,2787,2787
2020,1241,1666,1666,1666,1666,1665,1241,1666,1666,1666,...,1555,1666,1549,1666,1468,1463,1463,1666,1666,1666
2021,1247,1720,1720,1720,1720,1719,1219,1720,1720,1720,...,1665,1720,1657,1720,1490,1484,1484,1720,1720,1720


In [18]:
# Incomplete data for year(s) 2013 & 2023. Drop all data for those years.
results_df = results_df[results_df['yr_intake']>2013 & results_df['yr_intake']<2023]

# Verify successful
results_df['yr_intake'].unique()
results_df.groupby('yr_intake').count()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:


#results_df.info()